In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np

from numpy import ndarray
from copy import deepcopy

from typing import Callable, List, Tuple

# 06. RNN
In this chapter, we’ll cover `RNN`, a class of neural network architectures meant for handling sequences of data. The neural networks we’ve seen so far treated each batch of data they received as a set of independent observations; there was no notion of some of the MNIST digits arriving before or after the other digits, in either the fully connected neural networks we saw in `Chapter 4` or the `CNN` we saw in `Chapter 5`. Many kinds of data, however, are intrinsically ordered, whether time series data, which one might deal with in an industrial or financial context, or language data, in which the characters, words, sentences, and so on are ordered. `RNN` are designed to learn how to take in sequences of such data and return a correct prediction as output, whether that correct prediction is of the price of a financial asset on the following day or of the next word in a sentence.

Dealing with ordered data will require three kinds of changes from the fully connected neural networks we saw in the first few chapters:

First, it will involve "adding a new dimension" to the `ndarrays` we feed our neural networks. Previously, the data we fed our neural networks was intrinsically two-dimensional—each ndarray had one dimension representing the number of observations and another representing the number of features; another way to think of this is that each observation was a one-dimensional vector. With `RNN`, each input will still have a dimension representing the number of observations, but each observation will be represented as a two-dimensional `ndarray`: 
+ one dimension will represent the length of the sequence of data
+ a second dimension will represent the number of features present at each sequence element

The overall input to an `RNN` will thus be a three-dimensional ndarray of shape $[\textit{batch_size}, \textit{sequence_length}, \textit{num_features}]$—a batch of sequences.

Second, of course, to deal with this new three-dimensional input we’ll have to use a new kind of neural network architecture, which will be the main focus of this chapter. 

The third change, however, is where we’ll start our discussion in this chapter: we’ll have to use a completely different framework with different abstractions to deal with this new form of data. Why? In the cases of both fully connected and convolutional neural networks, each "operation", even if it in fact represented many individual additions and multiplications (as in the case of matrix multiplication or a convolution), could be described as a single "minifactory", that on both the forward and backward passes took in one `ndarray` as input and produced one `ndarray` as an output (possibly using another `ndarray` representing the operation’s parameters as part of these computations). As it turns out, `RNN` cannot be implemented in this way. Before reading further to find out why, take some time to think about it: what characteristics of a neural network architecture would cause the framework we’ve built so far to break down? While the answer is illuminating, the full solution involves concepts that go deep into implementation details and are beyond the scope of this book. To begin to unpack this, let’s reveal a key limitation of the framework we’ve been using so far.

## 6.1 The Key Limitation: Handling Branching
It turns out that our framework couldn’t train models with computational graphs like those depicted in `Figure 6-1`.

<img src="images/06_01.png" style="width:600px;"/>

What’s wrong with this? Translating the forward pass into code seems fine (note that we have written Add and Multiply operations here for illustration purposes only):